In [352]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.mlab import PCA

In [353]:
twitterBots = pd.read_csv('../data/bots_data.csv',encoding='latin-1')
twitterNonBots=pd.read_csv('../data/nonbots_data.csv',encoding='latin-1')
user=pd.read_csv('../data/training_data_2_csv_UTF.csv',encoding='latin-1')
test=pd.read_csv('../data/test_data_4_students.csv')
#dfList=[twitterBots,twitterNonBots]
#user = pd.concat(dfList)



user

In [354]:
df=user
#w['female'] = w['female'].map({'female': 1, 'male': 0})
df['favourites_count'].replace(['None'], '0')
df['default_profile'].replace(['None'], False)
df['default_profile_image'].replace(['None'], False)

#df['favourites_count']=np.where(pd.isnull(user['favourites_count']),0 ,df['favourites_count'])
#default_profile
#default_profile_image
#has_extended_profile
#


0       False
1       False
2       False
3        True
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11       True
12       True
13       True
14       True
15      False
16      False
17      False
18       True
19       True
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2767    False
2768    False
2769    False
2770    False
2771    False
2772    False
2773    False
2774    False
2775    False
2776    False
2777    False
2778    False
2779    False
2780    False
2781    False
2782    False
2783    False
2784    False
2785    False
2786    False
2787    False
2788     True
2789    False
2790    False
2791    False
2792    False
2793    False
2794    False
2795    False
2796    False
Name: default_profile_image, dtype: bool

In [355]:
tf = tf.rename(columns={'favorites_count': 'favourites_count'})
tf=test
#w['female'] = w['female'].map({'female': 1, 'male': 0})
tf['favorites_count']=tf['favorites_count'].replace(['None'], 0)
tf['default_profile']=tf['default_profile'].replace(['None'], False)
tf['default_profile_image']=tf['default_profile_image'].replace(['None'], False)
tf['favorites_count']=tf['favorites_count'].replace(['FALSE'], False)
tf['favorites_count']=tf['favorites_count'].replace(['TRUE'], True)
tf['favorites_count']=tf['favorites_count'].replace(['None'], False)
tf['default_profile']=tf['default_profile'].replace(['FALSE'], False)
tf['default_profile']=tf['default_profile'].replace(['TRUE'], True)
tf['default_profile']=tf['default_profile'].replace(['None'], False)
tf['default_profile_image']=tf['default_profile_image'].replace(['FALSE'], False)
tf['default_profile_image']=tf['default_profile_image'].replace(['TRUE'], True)
tf['default_profile_image']=tf['default_profile_image'].replace(['None'], False)
tf['verified']=tf['verified'].replace(['TRUE'], True)
tf['verified']=tf['verified'].replace(['FALSE'], False)
tf['verified']=tf['verified'].replace(['None'], False)
tf['followers_count']=tf['followers_count'].replace(['None'], 0)
tf['friends_count']=tf['friends_count'].replace(['None'], 0)
tf['statuses_count']=tf['statuses_count'].replace(['None'], 0)
#df['favourites_count']=np.where(pd.isnull(user['favourites_count']),0 ,df['favourites_count'])
#default_profile
#default_profile_image
#has_extended_profile
#



In [356]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df[['followers_count',
                                                        'friends_count','statuses_count','verified',

                                                        'favourites_count','default_profile','default_profile_image']],df['bot'] ,test_size=0.3, random_state=0)

In [357]:
testData=tf[['followers_count','friends_count','statuses_count','verified','favorites_count','default_profile','default_profile_image']]
testData=testData[:575]

In [358]:
clf=RandomForestClassifier(criterion='entropy',min_samples_split=10,min_samples_leaf=10)
model= clf.fit(X_train,Y_train)
predicted=model.predict(testData)
#print('Accuracy of RandomForest is',accuracy_score(Y_test,predicted)*100,'%')
#print(metrics.classification_report(Y_test,predicted))
#print('AUC Score for Random forest is',metrics.roc_auc_score(Y_test, predicted))

In [359]:
predicted


array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1,

In [360]:
af=test[:575]

In [361]:
af['bot']=predicted
nf=af[['id','bot']]

C:\Users\saura\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [363]:
nf.to_csv('test.csv', index = False)
nf['id']

0      2.281293e+09
1      2.344040e+09
2      7.658713e+08
3      4.772373e+09
4      1.324549e+09
5      2.561342e+09
6      3.478101e+08
7      8.563039e+08
8      8.328750e+17
9      8.885679e+07
10     7.135570e+17
11     1.566747e+09
12     9.042031e+07
13     1.849100e+08
14     1.576906e+08
15     4.242035e+07
16     4.238245e+07
17     4.399328e+07
18     2.305237e+09
19     7.427940e+17
20     3.134859e+07
21     1.220859e+08
22     2.357308e+07
23     4.315248e+07
24     1.888575e+08
25     2.911273e+09
26     3.509464e+07
27     1.462528e+08
28     8.543087e+07
29     5.511786e+07
           ...     
545    7.312010e+17
546    6.127548e+08
547    4.493562e+09
548    2.897137e+09
549    7.895600e+07
550    4.462343e+09
551    3.830053e+09
552    5.866719e+08
553    3.328881e+08
554    7.569370e+17
555    2.163813e+09
556    2.602313e+09
557    8.130000e+17
558    3.558834e+08
559    3.229507e+09
560    2.032293e+07
561    1.671255e+07
562    2.684399e+08
563    2.566952e+09
